<a href="https://colab.research.google.com/github/mgk0422/python-basic/blob/master/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 분류 성능평가 지표
# 정확도
# 오차행렬
# 정밀도
# 재현율
# F1 스코어
# ROC AUC

In [ ]:
# 정확도 : 직관적으로 모델 예측 성능을 나타내는 평가 지표
# 이진분류의 특성상 0이냐 1, 긍정/부정을 판단하는 이진분류 특성상 정확도 보다는 다른 성능 평가 지표가 중요하다

# 오차행렬 : 이진분류에서 활용 잘되고 학습된 분류 모델이 예측을 수행하면서 얼마나 헷갈리고 있는지도 함께 보여주는 지표

In [ ]:
import pandas as pd
import numpy as np
import glob
import sys
from openpyxl import load_workbook
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
    def fit(self,X,y=None):
        pass
    def predict(selt,X):
        pred=np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i]==1:
                pred[i]=0
            else:
                pred[i]=1
        return pred

In [ ]:
## 생성된 MyDummyClassifier를 이용해 타이타닉 생존자 예측 수행

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

## 머신러닝에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

## Label Encoding 수행
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

## 앞에서 실행한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
# 타이타닉 데이터 로딩 및 학습 데이터 / 테스트 데이터 분할
titanic_df = pd.read_excel('/content/sample_data/titanic_train.xlsx')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop(['Survived'], axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=0)

In [ ]:
myclf=MyDummyClassifier()
myclf.fit(X_train,y_train)

mypredictions=myclf.predict(X_test)
print('정확도{0:4f}'.format(accuracy_score(y_test,mypredictions)))

In [ ]:
# 불균형한 데이터 세트에서 정확도 보다 더 선호되는 평가 지표인 정밀도, 재현율에 대해 알아보자
# 정밀도 : 예측을 Positive로 한 대상 중에 예측과 실제값이 Positive로 일치한 데이터의 비율
# 재현율 : 실제 값이 Positive인 대상 중에 예측과 실제값이 Positive로 일치한 데이터의 비율

# 재현율이 중요한 지표인 경우 실제 Positive 양성 데이터를 Negative로 잘못 판단하게 되면 업무상 큰 영향이 발생하는 경우
# ex) 암, 보험사기(금융사기 적발모델)

# 정밀도가 더 중요한 지표인 경우
# ex) 스팸메일 벼우 판단

# 재현율과 정밀도는 모두 TP를 높이는데 동일하게 초점을 맞춘다.
# 가장 좋은 성능평가는 재현율과 정밀도 모두 높은 수치를 얻는것.

# 사이킷런에서 정밀도 계산 precision_score()
# 사이킷런에서 재현율 계산 recall_score()

In [ ]:
## 평가를 간편하게 적용하기 위해서는 confusion, matrix, accuracy, precision, recall 등 평가를 한꺼번에 호출하는 get_clf_eval() 만들것임

from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix

def get_clf_eval(y_test,pred):
    confusion=confusion_matrix(y_test,pred)
    accuracy=accuracy_score(y_test,pred)
    precision=precision_score(y_test,pred)
    recall=recall_score(y_test,pred)
    print('오차행렬')
    print(confusion)
    print('정확도:{0:.4f},정밀도:{1:.4f},재현율:{2:4f}'.format(accuracy,precision,recall))

In [ ]:
from sklearn.linear_model import LogisticRegression
titanic_df = pd.read_excel('/content/sample_data/titanic_train.xlsx')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop(['Survived'], axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)

lr_clf=LogisticRegression()
lr_clf.fit(X_train,y_train)
pred=lr_clf.predict(X_test)
get_clf_eval(y_test,pred)

In [ ]:
# 정밀도/재현율 트레이드오프
# 분류하려는 업무의 특성상 정밀도 또는 재현율이 특별히 강조돼야 할 경우 분류의 결정 임곗값을 조정해 정밀도 또는 재현율의 수치를 높일수있다
# 하지만 어느 한쪽을 강제적으로 높이면 다른 하나의 수치는 떨어지기 쉽다 -> 정밀도/재현율의 트레이드오프(Trade-off)

# 사이킷런 개별 데이터별로 예측 확률을 반환하는 메서드인 predict_proba()를 제공
# predict_proba() 메서드는 학습  완료된 사이킷런 Classifier 객체에서 호출이 가능하며 테스트 피처 데이터 세트를 파라미터로 입력해주면
# 테스트 피처 레코드의 개별 클래스 예측 확률을 반환

# 타이타닉  생존자 데이터에서 predict() 결과 값과 predict_proba() 결과 값을 비교
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)

print('pred_proba의 shape: {0}'.format(pred_proba.shape))
print('pred_proba의 array에서 앞 3개만 샘플로 추출 :\n', pred_proba[:3])

#예측확률 array와 예측 결과값 array를 병합하여 예측확률과 결괏값을 한 번에 확인
pred_proba_result = np.concatenate([pred_proba, pred.reshape(-1,1)], axis=1)
print('두 개의 class 중 더 큰 확률을 클래스 값으로 예측\n', pred_proba_result[:3])

In [ ]:
#Binarizer 클래스의 사용법
# 생성된 Binarizer 객체의 fit_transform() 메서드를 이용해 넘파이를 입력하면 입력된 넘파이의 값이 지정된 threshold보다
# 같거나 작으면 0값으로, 크면 1값으로 변환해 반환된다

In [ ]:
from sklearn.preprocessing import Binarizer
X=[[1,-1,2],[2,0,0],[0,1.1,1.2]] 

# X의 개별 원소들이 threshold값보다 같거나 작으면 0을 크면 1을 반환
binarizer=Binarizer(threshold=1.1)
print(binarizer.fit_transform(X))

In [ ]:
from sklearn.preprocessing import Binarizer

custom_threshold=0.5
pred_proba_1=pred_proba[:,1].reshape(-1,1)

binarizer=Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict=binarizer.transform(pred_proba_1)
get_clf_eval(y_test,custom_predict)

In [ ]:
# 임계값을 낮추면 평가지표는 어떻게 될까? 임곗값을 낮추니 재현율 값은 올라가고 정밀도가 떨어짐
# 분류 결정 임곗값은 Positive 예측값을 결정하는 확률의 기준이 된다.
# 확률이 0.5가 아닌 0.4부터 Positive로 예측을 더 너그럽게 하기때문에 임계값을 낮출수록 True값이 많아지게된다.

from sklearn.preprocessing import Binarizer

custom_threshold=0.4
pred_proba_1=pred_proba[:,1].reshape(-1,1)

binarizer=Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict=binarizer.transform(pred_proba_1)
get_clf_eval(y_test,custom_predict)

In [ ]:
# 임계값을 0.4에서부터 0.6까지 0.05씩 증가시키는 함수 생성 get_eval_by_threshold()

# 테스트 수행을 할 모든 임계값을 리스트로
thresholds=[0.4,0.45,0.5,0.55,0.6]

def get_eval_by_threshold(y_test,pred_proba_1,thresholds):
    for custom_threshold in thresholds:
        binarizer=Binarizer(threshold=custom_threshold).fit(pred_proba_1)
        custom_predict=binarizer.transform(pred_proba_1)
        print('임곗값 :',custom_threshold)
        get_clf_eval(y_test,custom_predict)

get_eval_by_threshold(y_test,pred_proba[:,1].reshape(-1,1),thresholds)